In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np

# Load Raw Data

In [3]:
data_dir = "../data/raw/results/"

In [4]:
years = [2014, 2016, 2018]
results_2014, results_2016, results_2018 = [
    pd.read_excel(data_dir + str(year) + "GeneralPrecinctResults.xlsx")
    for year in years
]

results = [results_2014, results_2016, results_2018]

In [6]:
results_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170145 entries, 0 to 170144
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   State                   170145 non-null  object
 1   Year                    170145 non-null  int64 
 2   Election Type           170145 non-null  object
 3   County                  170145 non-null  object
 4   Precinct                170145 non-null  int64 
 5   Office/Issue/Judgeship  170145 non-null  object
 6   Candidate               129377 non-null  object
 7   Party                   86795 non-null   object
 8   Candidate Votes         170145 non-null  int64 
 9   Yes Votes               170145 non-null  int64 
 10  No Votes                170145 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 14.3+ MB


In [9]:
results_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226031 entries, 0 to 226030
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   State                   226031 non-null  object 
 1   Year                    226031 non-null  int64  
 2   Election Type           226031 non-null  object 
 3   County                  226031 non-null  object 
 4   Precinct                226031 non-null  object 
 5   Office/Issue/Judgeship  226031 non-null  object 
 6   Candidate               197259 non-null  object 
 7   Party                   142762 non-null  object 
 8   Candidate Votes         226031 non-null  int64  
 9   Yes Votes               226021 non-null  float64
 10  No Votes                226021 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 19.0+ MB


In [10]:
results_2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148480 entries, 0 to 148479
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   State                   148480 non-null  object
 1   Year                    148480 non-null  int64 
 2   Election Type           148480 non-null  object
 3   County                  148480 non-null  object
 4   Precinct                148480 non-null  object
 5   Office/Issue/Judgeship  148480 non-null  object
 6   Candidate               136820 non-null  object
 7   Party                   97432 non-null   object
 8   Candidate Votes         148480 non-null  int64 
 9   Yes Votes               148480 non-null  int64 
 10  No Votes                148480 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 12.5+ MB


# Clean Data

# Extract Meaningful Variables

I want

"State Representative"
"State Senate"
"United States Representative"
"United States Senator"
"President/Vice President"

In [144]:
races = {"State Representative": "state_rep",
         "State Senate": "state_senate",
         "United States Representative": "us_rep",
         "United States Senator": "us_senate",
         "President/Vice President": "pres"}

In [147]:
def trim_to_relevant_races(df, races):
    drop_idxs = (df.loc[df["Office/Issue/Judgeship"]
                   .apply(lambda x: x.split('-')[0].strip())
                   .apply(lambda x: x not in races)].index)
    results_target = (df.drop(drop_idxs)
                        .drop(columns=["Yes Votes", "No Votes", "State", "Year", "Election Type", "County"])
                        .rename(columns={"Office/Issue/Judgeship": "Office"})
                        .assign(Office = lambda x: x.Office.apply(lambda x: races[x.split('-')[0].strip()])))
    return results_target

In [148]:
res_race_2014 = trim_to_relevant_races(results_2014, races)
res_race_2016 = trim_to_relevant_races(results_2016, races)
res_race_2018 = trim_to_relevant_races(results_2018, races)

# Find Significant Parties

In [128]:
def find_relevant_parties(df, office, threshold):
    party_vals = (df[df.Office == office]
                      .groupby("Party")
                      .sum()
                      .sort_values(by="Candidate Votes", ascending=False))
    party_vals["Candidate Votes"] = party_vals["Candidate Votes"]/party_vals["Candidate Votes"].sum()
    return party_vals[party_vals["Candidate Votes"] > threshold]

In [129]:
for target in target_vars:
    print(target)
    print(find_relevant_parties(res_target_2016, target, 0.001))
    print("\n")

State Representative
                   Candidate Votes
Party                             
Republican Party          0.492664
Democratic Party          0.478258
Libertarian Party         0.020334
Unaffiliated              0.008027


State Senate
                   Candidate Votes
Party                             
Democratic Party          0.534378
Republican Party          0.445282
Libertarian Party         0.020339


United States Representative
                   Candidate Votes
Party                             
Republican Party          0.476991
Democratic Party          0.467800
Libertarian Party         0.053076
Green Party               0.002088


United States Senator
                         Candidate Votes
Party                                   
Democratic Party                0.499707
Republican Party                0.443057
Libertarian Party               0.036192
Green Party                     0.013418
Unaffiliated                    0.004223
Unity Party of Colorado    

Okay, the parties I am interested in are those that have more than about a percent of the vote.

In [150]:
parties = {
    "Democratic Party": 'dem',
    "Republican Party": 'rep',
    "Libertarian Party": 'lib',
    "Green Party": 'grn',
    "Unaffiliated": 'unf'
}

In [155]:
def trim_to_relevant_parties(df, parties):
    drop_idx = df.loc[df["Party"].apply(lambda x: x not in parties)].index
    new_df = (df.drop(drop_idx, axis=0)
                .assign(Party = lambda x: x.Party.apply(lambda x: parties[x])))
    return new_df

In [156]:
res_race_party_2014 = trim_to_relevant_parties(res_race_2014, parties)
res_race_party_2016 = trim_to_relevant_parties(res_race_2016, parties)
res_race_party_2018 = trim_to_relevant_parties(res_race_2018, parties)

# Format Data by Precinct

In [176]:
def reshape_to_precinct_vals(df):
    final = (df.groupby(["Precinct", "Office", "Party"])
               .sum()
               .unstack(level=["Office","Party",])
               .fillna(0)
               .astype(int))
    final.columns = [race+" "+party for (cv, race, party) in final.columns]
    return final

In [177]:
res_final_2014 = reshape_to_precinct_vals(res_race_party_2014)
res_final_2016 = reshape_to_precinct_vals(res_race_party_2016)
res_final_2018 = reshape_to_precinct_vals(res_race_party_2018)

In [178]:
res_final_2018.head()

,state_rep dem,state_rep lib,state_rep rep,state_senate dem,state_senate lib,state_senate rep,us_rep dem,us_rep lib,us_rep rep,us_rep grn,state_rep unf,us_rep unf,state_senate unf
Precinct,,,,,,,,,,,,,
1160116112,489,35,259,506,20,263,498,37,266,0,0,0,0
1160116113,499,20,251,511,23,249,511,24,241,0,0,0,0
1160116114,557,33,284,566,28,288,581,30,276,0,0,0,0
1160116120,611,41,277,641,30,272,632,38,273,0,0,0,0
1160116122,506,35,373,524,19,380,528,22,370,0,0,0,0


# Save

In [179]:
res_final_2014.to_csv("../data/processed/results/results_2014.csv")
res_final_2016.to_csv("../data/processed/results/results_2016.csv")
res_final_2018.to_csv("../data/processed/results/results_2018.csv")